# ABOUT:
- In this notebook I:
    1. fine tuned **distilroberta-base** on the training set and evaluated on the validation set
- note:
    - without much manual extraction of features we can already achieve a rather high F1 Score of 0.952153

## main variables

In [1]:
model_checkpoint = "distilroberta-base"
num_labels = 2
batch_size = 30

## load_from_disk

In [2]:
from datasets import load_from_disk
tokenized_train_Dataset = load_from_disk(r"C:\Users\tanch\Documents\GitHub\Spam Detection (local)\data\tokenized_spam_Dataset")
tokenized_train_Dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 3900
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label'],
        num_rows: 1672
    })
})

In [3]:
from datasets import ClassLabel, Sequence
import pandas as pd
from IPython.display import display, HTML
def show_elements(dataset, randomize = True, num_samples = 10):
    
    if isinstance(dataset,pd.DataFrame):                  # if DataFrame 
        if randomize:                                          # if random> shuffle
            dataset = dataset.sample(frac=1)
        display(HTML(dataset.iloc[:num_samples].to_html()))             # take first n rows
    
    else:                                                    # if not DataFrame
        if randomize:                                           # if random> shuffle
            dataset = dataset.shuffle()   
        dataset = pd.DataFrame(dataset.select(range(num_samples)))   # convert first n rows to dataframe
        display(HTML(dataset.to_html()))
    

In [4]:
show_elements(tokenized_train_Dataset['train'])

Loading cached shuffled indices for dataset at C:\Users\tanch\Documents\GitHub\Spam Detection (local)\data\tokenized_spam_Dataset\train\cache-97b4ee15feb51edf.arrow


,attention_mask,input_ids,label
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1106, 47, 214, 45, 11, 127, 512, 11, 41, 1946, 8, 10, 457, 38, 437, 164, 6256, 4891, 405, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 30086, 1615, 5193, 6, 16, 452, 18, 529, 1474, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 530, 449, 35, 43, 5278, 29, 7359, 19, 162, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
3,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 31535, 6, 38, 581, 486, 423, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 791, 267, 45570, 45851, 3034, 12502, 66, 19, 1100, 7, 6255, 36649, 8, 2242, 927, 8461, 6625, 4, 305, 41407, 4, 6002, 1517, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 734, 3945, 47, 11, 5, 8881, 116, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
6,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## AutoModelForSequenceClassification

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model  = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, 
                                                            num_labels=num_labels)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

In [6]:
print(model.device)
model.to("cuda")

cpu


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

## F1 Score
- the harmonic mean of precision and recall
    1. precision 
        - proportion of predicted positives(spam) that are correctly predicted
        - important here because we don't want the model to wrongly predict many normal emails
    2. recall
        - proportion of actual positives that a correctly predicted
        - important here because we want the model the predict as many spam emails as possible

In [7]:
from sklearn.metrics import f1_score,confusion_matrix
def compute_f1(output):
    f1 = f1_score(output.label_ids, np.argmax(output.predictions,axis=1))
    return {"f1":f1}
def return_confusion_matrix(output):
    return confusion_matrix(output.label_ids, np.argmax(output.predictions,axis=1))

## TrainingArguments, Trainer

In [8]:
eval_steps = (len(tokenized_train_Dataset['train'])/batch_size)/5

In [9]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "steps",   
    logging_strategy ="steps",
    save_strategy ="no",                         
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,                               # best model may not be the model at the end of training, thus this param enables us to save any best model during training
    metric_for_best_model = "f1",
    eval_steps = eval_steps,
)

In [10]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_Dataset['train'],           
    eval_dataset=tokenized_train_Dataset['test'],
    compute_metrics = compute_f1
)

In [11]:
import mlflow
mlflow.end_run()
trainer.train()

***** Running training *****
  Num examples = 3900
  Num Epochs = 1
  Instantaneous batch size per device = 30
  Total train batch size (w. parallel, distributed & accumulation) = 30
  Gradient Accumulation steps = 1
  Total optimization steps = 130


Step,Training Loss,Validation Loss,F1
26,No log,0.159405,0.000000
52,No log,0.050771,0.938679
78,No log,0.050729,0.952153
104,No log,0.045369,0.949881
130,No log,0.046775,0.945368


***** Running Evaluation *****
  Num examples = 1672
  Batch size = 30


<IPython.core.display.Javascript object>

Saving model checkpoint to test-glue\checkpoint-26
Configuration saved in test-glue\checkpoint-26\config.json
Model weights saved in test-glue\checkpoint-26\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1672
  Batch size = 30


...

<IPython.core.display.Javascript object>

Saving model checkpoint to test-glue\checkpoint-104
Configuration saved in test-glue\checkpoint-104\config.json
Model weights saved in test-glue\checkpoint-104\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1672
  Batch size = 30


<IPython.core.display.Javascript object>

Saving model checkpoint to test-glue\checkpoint-130
Configuration saved in test-glue\checkpoint-130\config.json
Model weights saved in test-glue\checkpoint-130\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-glue\checkpoint-78 (score: 0.9521531100478469).


TrainOutput(global_step=130, training_loss=0.12072231586162861, metrics={'train_runtime': 97.6786, 'train_samples_per_second': 39.927, 'train_steps_per_second': 1.331, 'total_flos': 192160654920000.0, 'train_loss': 0.12072231586162861, 'epoch': 1.0})

In [12]:
model.save_pretrained(r"C:\Users\tanch\Documents\GitHub\Spam Detection (local)\models\distilroberta tuned")

Configuration saved in C:\Users\tanch\Documents\GitHub\Spam Detection (local)\models\distilroberta tuned\config.json
Model weights saved in C:\Users\tanch\Documents\GitHub\Spam Detection (local)\models\distilroberta tuned\pytorch_model.bin


## confusion_matrix 

In [12]:
output = trainer.predict(tokenized_train_Dataset['train'])
return_confusion_matrix(output)

***** Running Prediction *****
  Num examples = 3900
  Batch size = 30


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

array([[3350,   14],
       [  24,  512]], dtype=int64)

In [13]:
output = trainer.predict(tokenized_train_Dataset['test'])
return_confusion_matrix(output)

***** Running Prediction *****
  Num examples = 1672
  Batch size = 30


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

array([[1452,    9],
       [   9,  202]], dtype=int64)